In [1]:
import pandas as pd

# 讀取 CSV 檔案到資料框中
df = pd.read_csv('0520_no_label_data.csv')

# 顯示資料框的前幾行
df

,stock_symbol,title,date,paragraphs
0,TTWO,Take-Two Interactive Software (TTWO) Q4 2024 E...,2024-05-16,"Founded in 1993, The Motley Fool is a financia..."
1,AMAT,Applied Materials (AMAT) Q2 2024 Earnings Call...,2024-05-16,"Founded in 1993, The Motley Fool is a financia..."
2,DXC,DXC Technology (DXC) Q4 2024 Earnings Call Tra...,2024-05-16,"Founded in 1993, The Motley Fool is a financia..."
3,LSPD,Lightspeed Commerce (LSPD) Q4 2024 Earnings Ca...,2024-05-16,"Founded in 1993, The Motley Fool is a financia..."
4,JD,JD.com (JD) Q1 2024 Earnings Call Transcript,2024-05-16,"Founded in 1993, The Motley Fool is a financia..."
...,...,...,...,...
993,PYCR,Paycor HCM (PYCR) Q1 2024 Earnings Call Transc...,2023-11-08,"Founded in 1993, The Motley Fool is a financia..."
994,DUOL,Duolingo (DUOL) Q3 2023 Earnings Call Transcript,2023-11-08,"Founded in 1993, The Motley Fool is a financia..."
995,AFRM,Affirm (AFRM) Q1 2024 Earnings Call Transcript,2023-11-08,"Founded in 1993, The Motley Fool is a financia..."
996,CART,Instacart (CART) Q3 2023 Earnings Call Transcript,2023-11-08,"Founded in 1993, The Motley Fool is a financia..."


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# 設定 DataFrame 的欄位名稱
columns = ['stock_symbol', 'title', 'date', 
           '0_day_close', '1_day_close', '2_day_close', '3_day_close', 
           '4_day_close', '5_day_close', '7_day_close', '14_day_close', 
           '1_month_close', '2_month_close']

# 初始化新的 DataFrame
new_df = pd.DataFrame(columns=columns)

# 自定義函數來抓取股價資料並計算指定日期的收盤價
def get_close_prices(ticker, date):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=date, end=(datetime.strptime(date, "%Y-%m-%d") + timedelta(days=60)).strftime("%Y-%m-%d"))
    
    close_prices = {}
    if date in hist.index:
        close_prices['0_day_close'] = hist.loc[date]['Close']
        # 檢查每個日期是否存在於資料中，否則返回 None
        close_prices['1_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 1]]['Close'] if (hist.index.get_loc(date) + 1) < len(hist.index) else None
        close_prices['2_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 2]]['Close'] if (hist.index.get_loc(date) + 2) < len(hist.index) else None
        close_prices['3_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 3]]['Close'] if (hist.index.get_loc(date) + 3) < len(hist.index) else None
        close_prices['4_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 4]]['Close'] if (hist.index.get_loc(date) + 4) < len(hist.index) else None
        close_prices['5_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 5]]['Close'] if (hist.index.get_loc(date) + 5) < len(hist.index) else None
        close_prices['7_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 7]]['Close'] if (hist.index.get_loc(date) + 7) < len(hist.index) else None
        close_prices['14_day_close'] = hist.loc[hist.index[hist.index.get_loc(date) + 14]]['Close'] if (hist.index.get_loc(date) + 14) < len(hist.index) else None
        
        # 計算一個月和兩個月後的日期
        one_month_date = (datetime.strptime(date, "%Y-%m-%d") + timedelta(days=30)).strftime("%Y-%m-%d")
        two_month_date = (datetime.strptime(date, "%Y-%m-%d") + timedelta(days=60)).strftime("%Y-%m-%d")
        close_prices['1_month_close'] = hist.loc[one_month_date]['Close'] if one_month_date in hist.index else None
        close_prices['2_month_close'] = hist.loc[two_month_date]['Close'] if two_month_date in hist.index else None
    else:
        close_prices = {key: None for key in ['0_day_close', '1_day_close', '2_day_close', '3_day_close', 
                                               '4_day_close', '5_day_close', '7_day_close', '14_day_close', 
                                               '1_month_close', '2_month_close']}
    return close_prices

# 遍歷每一行並計算收盤價
for index, row in df.iterrows():
    date_str = row['date']
    ticker_symbol = row['stock_symbol']
    close_prices = get_close_prices(ticker_symbol, date_str)
    
    # 添加到新 DataFrame
    new_row = pd.DataFrame([{
        'stock_symbol': ticker_symbol,
        'title': row['title'],
        'date': date_str,
        **close_prices
    }])
    new_df = pd.concat([new_df, new_row], ignore_index=True)

# 顯示新 DataFrame
new_df

$CDAY: possibly delisted; No timezone found
$PUBL: possibly delisted; No timezone found
$TRMR: possibly delisted; No timezone found
$CDAY: possibly delisted; No timezone found
$PUBL: possibly delisted; No timezone found
$TRMR: possibly delisted; No timezone found
$NCR: possibly delisted; No timezone found
$NVTA: possibly delisted; No timezone found


,stock_symbol,title,date,time,0_day_close,1_day_close,2_day_close,3_day_close,4_day_close,5_day_close,7_day_close,14_day_close,1_month_close,2_month_close
0,TTWO,Take-Two Interactive Software (TTWO) Q4 2024 E...,2024-05-16,None,146.080002,147.839996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,AMAT,Applied Materials (AMAT) Q2 2024 Earnings Call...,2024-05-16,None,214.029999,212.080002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,DXC,DXC Technology (DXC) Q4 2024 Earnings Call Tra...,2024-05-16,None,19.879999,16.520000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,LSPD,Lightspeed Commerce (LSPD) Q4 2024 Earnings Ca...,2024-05-16,None,15.100000,15.420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,JD,JD.com (JD) Q1 2024 Earnings Call Transcript,2024-05-16,None,34.270000,35.270000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,PYCR,Paycor HCM (PYCR) Q1 2024 Earnings Call Transc...,2023-11-08,None,17.850000,18.639999,18.980000,19.610001,20.360001,20.730000,20.270000,21.340000,21.25,None
994,DUOL,Duolingo (DUOL) Q3 2023 Earnings Call Transcript,2023-11-08,None,167.270004,203.020004,211.529999,213.320007,219.309998,205.059998,211.649994,219.509995,214.75,None
995,AFRM,Affirm (AFRM) Q1 2024 Earnings Call Transcript,2023-11-08,None,21.760000,24.860001,22.430000,22.879999,25.559999,25.980000,24.790001,34.119999,42.07,None
996,CART,Instacart (CART) Q3 2023 Earnings Call Transcript,2023-11-08,None,27.240000,24.480000,25.209999,25.559999,25.879999,25.900000,25.250000,25.510000,24.440001,None


In [9]:
df['paragraphs']

0      Founded in 1993, The Motley Fool is a financia...
1      Founded in 1993, The Motley Fool is a financia...
2      Founded in 1993, The Motley Fool is a financia...
3      Founded in 1993, The Motley Fool is a financia...
4      Founded in 1993, The Motley Fool is a financia...
                             ...                        
993    Founded in 1993, The Motley Fool is a financia...
994    Founded in 1993, The Motley Fool is a financia...
995    Founded in 1993, The Motley Fool is a financia...
996    Founded in 1993, The Motley Fool is a financia...
997    Founded in 1993, The Motley Fool is a financia...
Name: paragraphs, Length: 998, dtype: object

In [11]:
new_df['paragraphs'] = df['paragraphs']
new_df = new_df.drop(columns=['time'])
new_df

,stock_symbol,title,date,0_day_close,1_day_close,2_day_close,3_day_close,4_day_close,5_day_close,7_day_close,14_day_close,1_month_close,2_month_close,paragraphs
0,TTWO,Take-Two Interactive Software (TTWO) Q4 2024 E...,2024-05-16,146.080002,147.839996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"Founded in 1993, The Motley Fool is a financia..."
1,AMAT,Applied Materials (AMAT) Q2 2024 Earnings Call...,2024-05-16,214.029999,212.080002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"Founded in 1993, The Motley Fool is a financia..."
2,DXC,DXC Technology (DXC) Q4 2024 Earnings Call Tra...,2024-05-16,19.879999,16.520000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"Founded in 1993, The Motley Fool is a financia..."
3,LSPD,Lightspeed Commerce (LSPD) Q4 2024 Earnings Ca...,2024-05-16,15.100000,15.420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"Founded in 1993, The Motley Fool is a financia..."
4,JD,JD.com (JD) Q1 2024 Earnings Call Transcript,2024-05-16,34.270000,35.270000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"Founded in 1993, The Motley Fool is a financia..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,PYCR,Paycor HCM (PYCR) Q1 2024 Earnings Call Transc...,2023-11-08,17.850000,18.639999,18.980000,19.610001,20.360001,20.730000,20.270000,21.340000,21.25,None,"Founded in 1993, The Motley Fool is a financia..."
994,DUOL,Duolingo (DUOL) Q3 2023 Earnings Call Transcript,2023-11-08,167.270004,203.020004,211.529999,213.320007,219.309998,205.059998,211.649994,219.509995,214.75,None,"Founded in 1993, The Motley Fool is a financia..."
995,AFRM,Affirm (AFRM) Q1 2024 Earnings Call Transcript,2023-11-08,21.760000,24.860001,22.430000,22.879999,25.559999,25.980000,24.790001,34.119999,42.07,None,"Founded in 1993, The Motley Fool is a financia..."
996,CART,Instacart (CART) Q3 2023 Earnings Call Transcript,2023-11-08,27.240000,24.480000,25.209999,25.559999,25.879999,25.900000,25.250000,25.510000,24.440001,None,"Founded in 1993, The Motley Fool is a financia..."


In [12]:
new_df.to_csv('0520_price_volatility_ver2.csv', index=False)